In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Example Workflows with MAPS\n",
    "\n",
    "This notebook demonstrates more complex workflows using the Multi-Agent Processing System (MAPS).\n",
    "\n",
    "## Setup\n",
    "\n",
    "First, let's import the necessary modules and initialize the system components."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "# Add the parent directory to the path if running in a notebook\n",
    "notebook_path = os.path.abspath('')\n",
    "project_root = os.path.join(notebook_path, '..')\n",
    "if project_root not in sys.path:\n",
    "    sys.path.append(project_root)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import MAPS components\n",
    "from core.memory_system import MemorySystem\n",
    "from core.function_registry import FunctionRegistry\n",
    "from core.conversation_manager import ConversationManager\n",
    "\n",
    "from agents.data_engineer import DataEngineerAgent\n",
    "from agents.viz_specialist import VizSpecialistAgent\n",
    "from agents.memory_agent import MemoryAgent\n",
    "from agents.orchestrator import OrchestratorAgent\n",
    "\n",
    "from config.system_config import DATA_DIRS\n",
    "from utils.sample_data import create_customer_data\n",
    "\n",
    "# Create required directories\n",
    "for dir_path in DATA_DIRS.values():\n",
    "    os.makedirs(dir_path, exist_ok=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Initialize System Components\n",
    "\n",
    "Now let's initialize all the components of our system."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize core components\n",
    "memory_system = MemorySystem(persistence_path=DATA_DIRS[\"memory\"])\n",
    "function_registry = FunctionRegistry()\n",
    "conversation_manager = ConversationManager(memory_system, persistence_path=DATA_DIRS[\"conversations\"])\n",
    "\n",
    "# Initialize agents\n",
    "data_engineer = DataEngineerAgent(memory_system, function_registry)\n",
    "viz_specialist = VizSpecialistAgent(\n",
    "    memory_system, \n",
    "    function_registry,\n",
    "    output_dir=DATA_DIRS[\"visualizations\"]\n",
    ")\n",
    "memory_agent = MemoryAgent(memory_system, conversation_manager)\n",
    "\n",
    "# Initialize orchestrator\n",
    "orchestrator = OrchestratorAgent(\n",
    "    data_engineer,\n",
    "    viz_specialist,\n",
    "    memory_agent,\n",
    "    conversation_manager\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Helper function to display outputs nicely\n",
    "def process_and_display(query):\n",
    "    print(f\"\\nQuery: {query}\")\n",
    "    result = orchestrator.process_query(query)\n",
    "    print(f\"Response: {result['message']}\")\n",
    "    \n",
    "    # If there's a visualization, display it\n",
    "    if result['success'] and 'result' in result and result['result']:\n",
    "        if 'filepath' in result['result']:\n",
    "            from IPython.display import Image\n",
    "            display(Image(filename=result['result']['filepath']))\n",
    "            \n",
    "    return result"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Workflow 1: Customer Segmentation Analysis\n",
    "\n",
    "In this workflow, we'll analyze customer data to identify segments and patterns."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create or load sample customer data\n",
    "sample_file = os.path.join(DATA_DIRS[\"sample_datasets\"], \"notebook_customer_data.csv\")\n",
    "\n",
    "if not os.path.exists(sample_file) or os.path.getsize(sample_file) == 0:\n",
    "    # Create sample customer data\n",
    "    np.random.seed(42)\n",
    "    \n",
    "    n_rows = 1000\n",
    "    \n",
    "    segments = ['Premium', 'Standard', 'Basic']\n",
    "    countries = ['USA', 'UK', 'Canada', 'Australia', 'Germany', 'France', 'Japan']\n",
    "    \n",
    "    today = pd.to_datetime('2023-12-31')\n",
    "    \n",
    "    data = {\n",
    "        'customer_id': np.random.randint(10000, 99999, size=n_rows),\n",
    "        'age': np.random.randint(18, 80, size=n_rows),\n",
    "        'customer_segment': np.random.choice(segments, size=n_rows, p=[0.2, 0.5, 0.3]),\n",
    "        'country': np.random.choice(countries, size=n_rows),\n",
    "        'registration_date': [(today - pd.Timedelta(days=np.random.randint(1, 1500))).strftime('%Y-%m-%d') for _ in range(n_rows)],\n",
    "        'membership_years': np.random.randint(0, 10, size=n_rows),\n",
    "        'purchase_amount': np.round(np.random.exponential(scale=500, size=n_rows), 2),\n",
    "        'service_fee': np.round(np.random.uniform(10, 100, size=n_rows), 2),\n",
    "        'rating': np.round(np.random.uniform(1, 5, size=n_rows), 1)\n",
    "    }\n",
    "    \n",
    "    # Introduce some missing values\n",
    "    for col in ['age', 'purchase_amount', 'rating']:\n",
    "        mask = np.random.choice([True, False], size=n_rows, p=[0.05, 0.95])\n",
    "        data[col] = pd.Series(data[col])\n",
    "        data[col][mask] = None\n",
    "        \n",
    "    # Introduce a smaller number of missing values for customer_id\n",
    "    mask = np.random.choice([True, False], size=n_rows, p=[0.01, 0.99])\n",
    "    data['customer_id'] = pd.Series(data['customer_id'])\n",
    "    data['customer_id'][mask] = None\n",
    "    \n",
    "    df = pd.DataFrame(data)\n",
    "    \n",
    "    # Save to CSV\n",
    "    os.makedirs(os.path.dirname(sample_file), exist_ok=True)\n",
    "    df.to_csv(sample_file, index=False)\n",
    "    \n",
    "    print(f\"Sample customer data created and saved to {sample_file}\")\n",
    "else:\n",
    "    df = pd.read_csv(sample_file)\n",
    "    print(f\"Loaded existing sample data from {sample_file}\")\n",
    "\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 1: Load and Explore Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(f\"Load data from '{sample_file}'\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Show information about the columns in this dataset\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"What are the missing values in this dataset?\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 2: Clean and Transform Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Fill missing age values with the mean age\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Create a new column 'age_group' with categories 'Young' for < 30, 'Middle-aged' for 30-50, and 'Senior' for > 50\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Create a column 'total_spent' that adds the values from 'purchase_amount' and 'service_fee'\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Create a column 'is_loyal' that is True when membership_years > 2\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 3: Explore Customer Segments"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Show a bar chart of average total_spent by customer_segment\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Create a boxplot of total_spent by customer_segment\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Show a bar chart of average total_spent by age_group\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 4: Advanced Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Create a scatter plot of purchase_amount vs service_fee colored by customer_segment\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Show a heatmap of correlations between numeric columns\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Group the data by age_group and customer_segment and show average total_spent\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Show me a visualization that compares total_spent across different customer segments and age groups\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 5: Memory and Context"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Summarize what we've done so far\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"List all visualizations we've created\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Save a checkpoint of our current state\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Workflow 2: Starting a New Session\n",
    "\n",
    "Let's demonstrate how the system maintains context across sessions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Start a new session\n",
    "new_session_id = conversation_manager.start_new_session()\n",
    "print(f\"Started new session with ID: {new_session_id}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"What data are we working with?\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Show me the distribution of total_spent\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Create a new column 'spending_category' with 'Low' for total_spent < 300, 'Medium' for total_spent between 300 and 700, and 'High' for total_spent > 700\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"Show a count plot of spending_category by customer_segment\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Check Memory Across Sessions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"How many operations have we performed across all sessions?\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "process_and_display(\"What columns have we created in the data?\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "These examples demonstrate more advanced workflows with the MAPS system, including:\n",
    "\n",
    "1. Customer segmentation analysis\n",
    "2. Creating complex derived variables\n",
    "3. Advanced visualizations\n",
    "4. Maintaining context across different sessions\n",
    "\n",
    "The system provides a conversational interface to data processing and visualization tasks while maintaining context throughout the analysis process."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}